In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
data = pd.read_parquet('train-00000-of-00001-b21313e511aa601a.parquet')

# Drop rows with any missing values
data = data.dropna(ignore_index=True)

# Display the first 5 rows of the data
data.head()

,bibcode,title,abstract,verified_uat_ids,verified_uat_labels
0,2020ApJ...891..100S,Dynamic Potential Sputtering of Lunar Analog M...,"Pyroxenes ((Ca, Mg, Fe, Mn)<SUB>2</SUB>Si<SUB>...","[1534, 499, 1692, 948, 1024, 2004]","[solar wind, exosphere, the moon, lunar compos..."
1,2024ApJ...966L...8B,"Generation of Low-inclination, Neptune-crossin...",The solar system's distant reaches exhibit a w...,"[1705, 1184, 2293]","[trans-neptunian objects, orbits, solar system..."
2,2024PSJ.....5...45C,Leveraging the Gravity Field Spectrum for Icy ...,Understanding the interior structures of icy m...,"[2189, 1248, 770, 1889, 627, 1255]","[europa, planetary interior, hydrosphere, mark..."
3,2022ApJ...932...52H,Inverse Multiview. I. Multicalibrator Inverse ...,Very Long Baseline Interferometry (VLBI) astro...,"[1769, 1337, 1713, 1295]","[very long baseline interferometry, radio astr..."
4,2024ApJS..271...25C,The First LHAASO Catalog of Gamma-Ray Sources,We present the first catalog of very-high-ener...,"[628, 632, 205]","[gamma-ray astronomy, gamma-ray observatories,..."


After Loading the data, we're going to prepare the data

In [24]:
X = data[['title','abstract']]
Y = data['verified_uat_ids']

On va créer un dictionnaire avec les valeurs des ids vérifiés, associé à chaque label vérifié

In [25]:
ids_and_labels = data[['verified_uat_ids','verified_uat_labels']]

dict_ids_labels = {}

for i in range(len(ids_and_labels['verified_uat_ids'])):
    for j in range(len(ids_and_labels['verified_uat_ids'][i])):
        dict_ids_labels[ids_and_labels['verified_uat_ids'][i][j]] = ids_and_labels['verified_uat_labels'][i][j]

Create Y with columns equals to the value of the labels, with 1 or 0 to know if the label is the answer

In [ ]:
y_liste = []
for i in range(0,len(data['verified_uat_ids'])):
    for value in data['verified_uat_ids'][i]:
        for key in dict_ids_labels:
            if value == key:
                y_liste.append(dict_ids_labels[key])
print(y_liste)

We are going to separate our data into train and test datas

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

We are going to tokenize the text, so our model can handle it

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect= TfidfVectorizer()
X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)


We are now going to train our model

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

clf = MultiOutputClassifier(MultinomialNB()).fit(X_train_vect, Y_train)




ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()